In [1]:
#include <iostream>
#include <string>
#include <algoviz/SVG.hpp>
#include <cstdlib>

#include "Vogel.hpp"
#include "Layout.hpp"

using namespace std;

AlgoViz::clear();
SVG spielfeld = SVG(800,400);

Rect barriere[9];
Rect barriereZwei[9];

string key = "";

float gravity = 5;

int start = 50;
int runden = 0;
int rnd;
int rnd1;
int rnd2;
int y = 0;
int positionPressed;
int rndItemIndex = 2;
int rndYKoordinate = 200;
int tmpRunden;
int geschwindigkeit = 0;
int score;

bool reach400 = false;
bool crashed = false;
bool flyUp = false;

Layout layout = Layout(spielfeld);
Vogel vogel = Vogel(spielfeld);
Text screenScore;
Text finalScore;

In [2]:
void erstelleBarriere(Rect array[]) {  
    rnd = (rand() % 6);
     
    for (int block = 0; block < 9; block++ ) {
            // make two parts at index rnd(+1 & +2) of barrier transparent
            array[rnd].setFill(255, 255, 255, 0);
            array[rnd].setColor("transparent");
            array[rnd+1].setFill(255, 255, 255, 0);
            array[rnd+1].setColor("transparent");
            array[rnd+2].setFill(255, 255, 255, 0);
            array[rnd+2].setColor("transparent");
    
            // create blocks
            array[block] = Rect(800, 422-start, 50, 50, &spielfeld);
            array[block].setFill(0, 0, 0, 1);
            array[block].setColor("transparent");
        
             // block position -> y-axis
            start += 49;
    }
    start = 50;
}

In [3]:
void reseteBarriere(Rect array[], int rnd, int barrieren){
    // change color of previous transparent blocks to black
    array[rnd].setFill(0, 0, 0, 1);
    array[rnd].setColor("transparent");
    array[rnd+1].setColor("transparent");
    array[rnd+1].setFill(0, 0, 0, 1);
    array[rnd+2].setColor("transparent");
    array[rnd+2].setFill(0, 0, 0, 1);
    
    // generate new random missing block
    rnd = (rand() % 6);
    
    // which barrier to reset
    if (barrieren == 1) {
        rnd1 = rnd;
    } else {
        rnd2 = rnd;
    }
    
    // make two blocks transparent
    array[rnd].setFill(0, 0, 0, 0);
    array[rnd].setColor("transparent");
    array[rnd+1].setFill(0, 0, 0, 0);
    array[rnd+1].setColor("transparent");
    array[rnd+2].setFill(0, 0, 0, 0);
    array[rnd+2].setColor("transparent");     
}

In [4]:
void checkCollision (int KoordinateLowerLeftX, int KoordinateUpperLeftY){
    // rocket constantly x = 200 
    // check depending of block width 
        if (KoordinateLowerLeftX > 150 && KoordinateLowerLeftX < 225){
            if (KoordinateUpperLeftY+5 < vogel.getYKoordinate() && KoordinateUpperLeftY +150 > vogel.getYKoordinate()){
                score++;
            } else {
                crashed = true;
            }
        }
    
        // check upper & lower border
        if (vogel.getYKoordinate()> 400 || vogel.getYKoordinate() < 0){
            crashed = true;
        }
        
        if (crashed == true){
            // hide barriers
            for (int block = 0; block < 9; block++ ) {
                // zwei Steine transparent stellen
                barriere[block].setFill(0, 0, 0, 0);
                barriereZwei[block].setFill(0, 0, 0, 0);
                barriere[block].setColor(0, 0, 0, 0);
                barriereZwei[block].setColor(0, 0, 0, 0);
            }
            
            layout.flackern();
            layout.failedAttempt();
            vogel.vogelDead();
            finalScore = Text("Your final score: " + to_string(score), 360 , 100, &spielfeld);
            finalScore.setAttribute("stroke", "white");    
        }

}

In [5]:
void collision(Rect array1[], Rect array2[]){
    int KoordinateLowerLeftX;
    int KoordinateUpperLeftY;
    
    // which barrier is first to prove collision
    if (array1[rnd1].getX() < array2[rnd2].getX()){
        // get Koordinates of transpent blocks (free sgeschwindigkeit) from first barrier
        KoordinateLowerLeftX = array1[rnd1].getX();
        KoordinateUpperLeftY = array1[rnd1+2].getY();
        
        checkCollision(KoordinateLowerLeftX,KoordinateUpperLeftY);
    } else {
        // get Koordinates of transpent blocks (free sgeschwindigkeit) from second barrier
        KoordinateLowerLeftX = array2[rnd2].getX();
        KoordinateUpperLeftY = array2[rnd2 + 2].getY();
        
        checkCollision(KoordinateLowerLeftX,KoordinateUpperLeftY);
    }  
}

In [6]:
layout.hideWelcome();

while (crashed == false && key != "x" ) {
   key = spielfeld.lastKey();
    
    // spawn item every 190 runden
   if (runden > 2 && runden % 190 == 1){
        rndItemIndex = (rand() % 4);
        rndYKoordinate = (rand() % 400);
    }
    
    // create inital barrier
    if (runden == 0) {
        erstelleBarriere(barriere);
        rnd1 = rnd;
    }
 
    if (key == "ArrowUp") {
        y = y-40;
        vogel.move(y);
        flyUp = true;
        positionPressed = vogel.getYKoordinate();
    }

    collision(barriere, barriereZwei);
    

    // rotate rocket based on gravity 
    if (gravity < 0){
        vogel.rotate(-25);
    } else {
        vogel.rotate(45);
    }
    
    // change gravity
    if (flyUp == true) {
        gravity = gravity - 20; 
        // threshold gravity    
        if (gravity < -20){
            gravity = -20;
        }
        
        y += gravity;
        
        vogel.move(y);
        
        // switch direction
        if((vogel.getYKoordinate() - positionPressed) < 15) {
            flyUp = false;
        }
    } else {
        // maximal gravity 
        gravity = gravity + 1.5;   
            if (gravity > 12){
                gravity = 7;
            }

        y += gravity;

        vogel.move(y);
    }
    
    // create second barrier when first barrier reaches 400 (x-axis)
    if (barriere[0].getX() == 400 && reach400 == false) {
        reach400 = true;
        erstelleBarriere(barriereZwei);
        rnd2 = rnd; 
    }
    
    // reset barrier
    if (barriere[0].getX() == 0) {
        reseteBarriere(barriere, rnd1, 1);
    } else if (barriereZwei[0].getX() == 0) {
        reseteBarriere(barriereZwei, rnd2, 2);
    }
    
    // reset geschwindigkeit after 80 rounds
    if (runden - tmpRunden > 80){
        geschwindigkeit = 0;
    }
    
    // constantly move barriers
    for (int k = 0; k < 9; k++){
       barriere[k].moveTo((barriere[k].getX() - 10 + geschwindigkeit + 850) % 850, barriere[k].getY()); 
       barriereZwei[k].moveTo((barriereZwei[k].getX() - 10 + geschwindigkeit + 850) % 850, barriereZwei[k].getY());
    }
    

    screenScore = Text("Score: " + to_string(score), 10, -5, &spielfeld);

    
    runden++;
}